In [86]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks, butter, filtfilt
from statsmodels import api
def lowpass_filter(signal, cutoff_freq, fs, order=5):
	# lowpass filter
	b, a = butter(order, cutoff_freq / (fs / 2), btype='lowpass')
	filtered_signal = filtfilt(b, a, signal)

	return filtered_signal


class HumanPrescenceClassifier:
    def __init__(self, snr_threshold, motion_threshold, prominence=0.011, width=70, sampling_freq=100):
        self.snr_threshold = snr_threshold
        self.motion_threshold = motion_threshold
        self.prominence = prominence
        self.width = width
        self.sampling_freq = sampling_freq


    def maximum_ratio_combining(self):
        "Return weights under Maximum Ratio Combining"
        # use Maximum Ratio Combining
        mss = []
        # calculate the motion statistics from all subcarriers
        for i in range(self.subcarrier_power.shape[1]):
            CSI_amplitude = self.subcarrier_power[:, i]
            autocorr = api.tsa.stattools.acf(CSI_amplitude, nlags=100 - 1)
            mss.append(autocorr[1])

        # use Maximum Ratio Combining
        mss = np.array(mss)
        mss = mss / np.sum(mss)
        return mss
    

    def mrc_csi(self):
        weights = self.maximum_ratio_combining()
        CSI_full = self.subcarrier_power * weights
        CSI_full = np.sum(CSI_full, axis=1)
        return CSI_full
    
    def motion_detection_score_sub1(self, subcarrier=1):
        # CSI_amplitude = np.abs(CSI_raw)
        # CSI_amplitude = gaussian_filter1d(CSI_amplitude, 3)**2
        CSI_amplitude = self.subcarrier_power[:, subcarrier]
        
        autocorr = api.tsa.stattools.acf(CSI_amplitude, nlags=500 - 1)
        motion_statistics = autocorr[1]
        return motion_statistics

    
    def preprocess(self, csi_data):
        """csi_data not processed"""
        # Assume that the CSI data is stored in a 2D numpy array called csi_data,
        # where each row represents a time sample and each column represents a subcarrier.

        # Identify the subcarriers to prune (e.g., subcarriers with all 0 values)
        subcarriers_to_prune = np.where(~csi_data.any(axis=0))[0]

        # Remove the subcarriers from the data
        pruned_csi_data = np.delete(csi_data, subcarriers_to_prune, axis=1)

        # Verify the results
        # print("Original CSI data size:", csi_data.shape)
        # print("Pruned CSI data size:", pruned_csi_data.shape)

        csi_data = pruned_csi_data

        if csi_data.shape[1] == 0:
            raise ValueError("No subcarriers left after pruning")


        # Assume that the CSI data is stored in a numpy array called csi_data.
        # The CSI data should be a 2D array with shape (num_samples, num_subcarriers).

        # Compute the power of each subcarrier
        subcarrier_power = np.square(np.abs(csi_data))

        # Compute the noise power by averaging the power over all samples
        noise_power = np.mean(subcarrier_power, axis=1)

        # Compute the SNR of each subcarrier by dividing its power by the noise power
        subcarrier_snr = subcarrier_power / noise_power[:, np.newaxis]

        # # Set a threshold for the minimum SNR value
        # snr_threshold = 0.01

        # Find the subcarriers with SNR values above the threshold
        good_subcarriers = np.where(np.all(subcarrier_snr >= self.snr_threshold, axis=0))[0]

        if len(good_subcarriers) == 0:
            self.subcarrier_power = subcarrier_power
            return 0

        # Filter out the subcarriers with low SNR
        filtered_csi_data = subcarrier_power[:, good_subcarriers]

        # find the best subcarrier
        best_subcarrier = np.argmax(np.mean(filtered_csi_data, axis=0))

        # print("Best subcarrier:", best_subcarrier)
        
        # print("Original CSI data size:", subcarrier_power.shape)
        # print("Filtered CSI data size:", filtered_csi_data.shape)
        self.subcarrier_power = filtered_csi_data

        return best_subcarrier

    def predict(self, csi_data):
        # Filter out the subcarriers with low SNR
        best_subcarrier = self.preprocess(csi_data)
        # print("Best subcarrier:", best_subcarrier)

        # # Compute the motion statistics for each subcarrier
        # motion_statistics = motion_detection_score_all(filtered_CSI_amplitude)

        # # Find the subcarrier with the highest motion statistics
        # best_subcarrier = np.argmax(motion_statistics)

        # Compute the motion statistics for the best subcarrier
        motion_statistics = self.motion_detection_score_sub1(subcarrier=best_subcarrier)

        if motion_statistics >= self.motion_threshold:
            return True
        
        
        autocorr = lowpass_filter(api.tsa.stattools.acf(self.mrc_csi(), nlags=700 - 1)[1:], 1, self.sampling_freq)

        autocorr *= -1
        # Find the peaks in the motion statistics
        peaks, _ = find_peaks(autocorr, prominence=self.prominence, width=self.width)

        return len(peaks) >= 2
    
    
def parse_data(data_string):
    return data_string[1:-1].split(',')

def parse_data2(data_string):
    return data_string[1:-1].split(', ')

def parse_csi_data(data):
    return [complex(int(data[i]), int(data[i+1])) for i in range(0, len(data), 2)]

def parse_csi_data2(data):
    return [complex(int(data[i][1:-1]), int(data[i+1][1:-1])) for i in range(0, len(data), 2)]


In [87]:
# df = pd.read_csv('go_outside/3.csv')

In [88]:
classifier = HumanPrescenceClassifier(snr_threshold=0.009, motion_threshold=0.1, prominence=0.011, width=70, sampling_freq=100)

In [89]:
# combine the data
file_list_positive = ['sitting/2.csv', 'sitting/3.csv', 'sitting/4.csv', 'sitting/csi_data.csv', 'walking/1.csv', 'walking/2.csv', 'walking/3.csv', 'walking/4.csv', 'walking/5.csv'], (parse_data, parse_csi_data), True
file_list_negative = ['no one/2.csv', 'no one/3.csv', 'no one/csi_data.csv'], (parse_data2, parse_csi_data2), False
CSI_data_list = []
labels = []

window_size = 1000
step_size = 500

for file_list, (f1, f2), label in file_list_positive, file_list_negative:
    data = []
    for file in file_list:
        df = pd.read_csv(file)
        df['data'] = df['data'].apply(f1)
        CSI_data = np.array(df['data'].apply(f2).to_list())

        for i in range(window_size, CSI_data.shape[0], step_size):
            # get the window data (last window size data)
            window_CSI_data = CSI_data[max(0, i-window_size):i, :]
            
            # predict
            data.append(window_CSI_data)
        print("----")

    data = np.array(data[:])
    CSI_data_list.append(data)
    print(data.shape)
    # labels.append(np.ones(CSI_data.shape[0]) * label)
    labels.append(np.full((data.shape[0],), label, dtype=bool))


CSI_data = np.concatenate(CSI_data_list, axis=0)
print(CSI_data.shape)
labels = np.concatenate(labels, axis=0)
print(labels.shape)

# CSI_data = np.concatenate(CSI_data_list, axis=0)
# print(CSI_data.shape)
# labels = np.concatenate(labels, axis=0)
# print(labels.shape)


# for file_list, (f1, f2), label in file_list_positive, file_list_negative:
#     df = None
#     for file in file_list:
#         if df is None:
#             df = pd.read_csv(file)
#         else:
#             df = df.append(pd.read_csv(file))

#         # df = pd.read_csv(file)
#     df['data'] = df['data'].apply(f1)
#     CSI_data = np.array(df['data'].apply(f2).to_list())
#     CSI_data_list.append(CSI_data)
#     print(CSI_data.shape)
#     # labels.append(np.ones(CSI_data.shape[0]) * label)
#     labels.append(np.full((CSI_data.shape[0],), label, dtype=bool))

# CSI_data = np.concatenate(CSI_data_list, axis=0)
# print(CSI_data.shape)
# labels = np.concatenate(labels, axis=0)
# print(labels.shape)
# window_size = 1000
# step_size = 500
# for i in range(window_size, CSI_data.shape[0], step_size):
#     # get the window data (last window size data)
#     window_CSI_data = CSI_data[max(0, i-window_size):i+1, :]
    
#     # predict
#     if not classifier.predict(window_CSI_data):
#         print("No human prescence detected at index", i)
# print("----")


----
----
----
----
----
----
----
----
----
(39, 1000, 64)
----
----
----
(11, 1000, 64)
(50, 1000, 64)
(50,)


In [90]:
labels[-10:]

array([False, False, False, False, False, False, False, False, False,
       False])

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.optimize import minimize
from bayes_opt import BayesianOptimization

# Assume that the CSI data is stored in a numpy array called csi_data.
# The CSI data should be a 2D array with shape (num_samples, num_subcarriers).

# Split the CSI data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(CSI_data, labels, test_size=.3, random_state=42)

In [92]:
X_val.shape

(15, 1000, 64)

In [93]:
X_val.shape

(15, 1000, 64)

In [94]:
classifier = HumanPrescenceClassifier(snr_threshold=0.1, motion_threshold=0.1, prominence=0.011, width=70, sampling_freq=100)

In [95]:

classifier = HumanPrescenceClassifier(**{'motion_threshold': 0.27507400608839916, 'prominence': 0.021202887303199747, 'snr_threshold': 0.035990422909134456, 'width': 34.657920510962626})

In [96]:
def report_metrics(y_true, model, X):
    # predict valid set
    predictions = []

    for i in range(X.shape[0]):
        # get the window data (last window size data)
        window_CSI_data = X[i]
        
        # predict
        predictions.append(model.predict(window_CSI_data))

    # report the tpr, fpr, tnr, fnr
    print("TPR:", np.sum(np.logical_and(predictions, y_true)) / np.sum(y_true))
    print("FPR:", np.sum(np.logical_and(predictions, np.logical_not(y_true))) / np.sum(np.logical_not(y_true)))
    print("TNR:", np.sum(np.logical_and(np.logical_not(predictions), np.logical_not(y_true))) / np.sum(np.logical_not(y_true)))
    print("FNR:", np.sum(np.logical_and(np.logical_not(predictions), y_true)) / np.sum(y_true))

    print("Accuracy:", accuracy_score(y_true, predictions))
    print("Precision:", precision_score(y_true, predictions))
    print("Recall:", recall_score(y_true, predictions))
    print("F1:", f1_score(y_true, predictions))


In [97]:
best_classifier = HumanPrescenceClassifier(snr_threshold=0.1, motion_threshold=0.1, prominence=0.011, width=70, sampling_freq=100)

In [98]:
report_metrics(y_val, best_classifier, X_val)

TPR: 0.9166666666666666
FPR: 0.0
TNR: 1.0
FNR: 0.08333333333333333
Accuracy: 0.9333333333333333
Precision: 1.0
Recall: 0.9166666666666666
F1: 0.9565217391304348


In [99]:
report_metrics(labels, best_classifier, CSI_data)

TPR: 0.9230769230769231
FPR: 0.0
TNR: 1.0
FNR: 0.07692307692307693
Accuracy: 0.94
Precision: 1.0
Recall: 0.9230769230769231
F1: 0.9600000000000001


In [100]:
# Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

# Define the function to optimize, which takes two threshold values and returns the F1 score
def objective(snr_threshold, motion_threshold, prominence, width):
    predictions = []
    classifier = HumanPrescenceClassifier(snr_threshold=snr_threshold, motion_threshold=motion_threshold, prominence=prominence, width=width, sampling_freq=100)
    for i in range(X_train.shape[0]):
        # get the window data (last window size data)
        window_CSI_data = X_train[i]
        
        # predict
        predictions.append(classifier.predict(window_CSI_data))

    score = f1_score(y_train, predictions)  # Compute the F1 score
    return score  # Minimize the negative F1 score

# Set the bounds of the search space for the thresholds
pbounds = {'snr_threshold': (0, 1), 'motion_threshold': (0, 1), 'prominence': (0, 1), 'width': (0, 100)}

# Use a Bayesian optimization algorithm to find the optimal thresholds
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=10,
    n_iter=15,
)

# Print the optimal thresholds and the maximum F1 score
print('Optimal thresholds:', optimizer.max['params'])
print('Maximum accuracy_score score:', optimizer.max['target'])
classifier = HumanPrescenceClassifier(**optimizer.max['params'])
# predict valid set
predictions = []

for i in range(X_val.shape[0]):
    # get the window data (last window size data)
    window_CSI_data = X_val[i]
    
    # predict
    predictions.append(classifier.predict(window_CSI_data))
print("Validation accuracy_score:", f1_score(y_val, predictions))

|   iter    |  target   | motion... | promin... | snr_th... |   width   |
-------------------------------------------------------------------------
| 1         | 0.8261    | 0.417     | 0.7203    | 0.0001144 | 30.23     |
| 2         | 0.875     | 0.1468    | 0.09234   | 0.1863    | 34.56     |
| 3         | 0.8511    | 0.3968    | 0.5388    | 0.4192    | 68.52     |
| 4         | 0.875     | 0.2045    | 0.8781    | 0.02739   | 67.05     |
| 5         | 0.8261    | 0.4173    | 0.5587    | 0.1404    | 19.81     |
| 6         | 0.3636    | 0.8007    | 0.9683    | 0.3134    | 69.23     |
| 7         | 0.1379    | 0.8764    | 0.8946    | 0.08504   | 3.905     |
| 8         | 0.875     | 0.1698    | 0.8781    | 0.09835   | 42.11     |
| 9         | 0.0       | 0.9579    | 0.5332    | 0.6919    | 31.55     |
| 10        | 0.7442    | 0.6865    | 0.8346    | 0.01829   | 75.01     |
| 11        | 0.871     | 0.00772   | 0.1619    | 0.7094    | 76.47     |
| 12        | 0.8261    | 0.4426    | 